### TRINH THE SON - 20127617

In [2]:
import datetime
import numpy as np
import os
import pandas as pd
import shutil
import urllib.request
import zipfile

In [1]:
VARIANTS = {
    "100k": {"filename": "u.data", "sep": "\t"},
    "1m": {"filename": "ratings.dat", "sep": r"::"},
    "20m": {"filename": "ratings.csv", "sep": ","},
}

In [7]:
variant = "100k"

if variant not in VARIANTS:
    raise ValueError(
        f"Invalid variant: {variant}. Valid options are {list(VARIANTS.keys())}"
    )

In [10]:
url = f"http://files.grouplens.org/datasets/movielens/ml-{variant}.zip"

In [13]:
variant_info = VARIANTS[variant]
filename = variant_info["filename"]
dirname = f"ml-{variant}"
zip_path = os.path.join(dirname + ".zip")

In [16]:
with urllib.request.urlopen(url) as r, open(zip_path, "wb") as f:
    shutil.copyfileobj(r, f)
with zipfile.ZipFile(zip_path, "r") as zf:
    zf.extractall()

In [17]:
# Remove zip file after extraction (optional)
os.remove(zip_path)

FileNotFoundError: [Errno 2] No such file or directory: 'ml-100k.zip'

In [18]:
csv_path = os.path.join(dirname, filename)

In [27]:
names = ["u_id", "i_id", "rating", "timestamp"]
dtype = {"u_id": np.uint32, "i_id": np.uint32, "rating": np.float64}

df = pd.read_csv(
    csv_path,
    names=names,
    dtype=dtype,
    header=0,
    sep=VARIANTS[variant]["sep"],
)

df.sort_values(by="u_id", inplace=True)
df.reset_index(drop=True, inplace=True)

In [31]:
train = df.sample(frac=0.8, random_state=7)
val = df.drop(train.index.tolist()).sample(frac=0.5, random_state=8)
test = df.drop(train.index.tolist()).drop(val.index.tolist())

In [56]:
# def fetch_ml_ratings(variant="100k"):
#     VARIANTS = {
#         "100k": {"filename": "u.data", "sep": "\t"},
#         "1m": {"filename": "ratings.dat", "sep": r"::"},
#         "20m": {"filename": "ratings.csv", "sep": ","},
#     }

#     if variant not in VARIANTS:
#         raise ValueError(
#             f"Invalid variant: {variant}. Valid options are {list(VARIANTS.keys())}"
#         )

#     variant_info = VARIANTS[variant]
#     url = f"http://files.grouplens.org/datasets/movielens/ml-{variant}.zip"
#     filename = variant_info["filename"]
#     dirname = f"ml-{variant}"
#     zip_path = os.path.join(dirname + ".zip")

#     # Download and extract data
#     print("Downloading ml-" + variant + " and unzipping ...")
#     with urllib.request.urlopen(url) as r, open(zip_path, "wb") as f:
#         shutil.copyfileobj(r, f)
#     with zipfile.ZipFile(zip_path, "r") as zf:
#         zf.extractall()

#     # Remove zip file after extraction (optional)
#     os.remove(zip_path)

#     # Construct data path based on variant
#     csv_path = os.path.join(dirname, filename)

#     # Load data using separate function (not shown here)
#     df = _ml_ratings_csv_to_df(csv_path, variant)

#     return df

In [51]:
# def _ml_ratings_csv_to_df(csv_path, variant):
#     names = ["u_id", "i_id", "rating", "timestamp"]
#     dtype = {"u_id": np.uint32, "i_id": np.uint32, "rating": np.float64}

#     def date_format(time):
#         return datetime.datetime.fromtimestamp(float(time))

#     df = pd.read_csv(
#         csv_path,
#         names=names,
#         dtype=dtype,
#         header=0,
#         sep=VARIANTS[variant]["sep"],
#         parse_dates=["timestamp"],
#         date_format=date_format,
#         engine="python",
#     )

#     df.sort_values(by="timestamp", inplace=True)
#     df.reset_index(drop=True, inplace=True)

#     return df